In [1]:
# Objetivo: Encontrar categorias de crimes

In [2]:
import numpy as np
import pandas as pd
import itertools
from datetime import datetime, time
from sklearn.preprocessing import OneHotEncoder
import json

import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.path as path
from datetime import datetime
%matplotlib inline

## Import e tratamentos

In [3]:
def trata_vars(df):
    df['Hour'] = df.Dates.dt.hour.astype('str')
    df['Month'] = df.Dates.dt.month.astype('str')
    df['Year'] = df.Dates.dt.year.astype('str')
    df['Day'] = df.Dates.dt.day.astype('str')
    df['YearMonth'] = df.Dates.dt.year.astype('str') + df.Dates.dt.month.astype('str')
    df['YearDistrict'] = df.Dates.dt.year.astype('str') + df.PdDistrict
    df['esquina'] = df.Address.str.contains('/').astype('int')
    df['MonthDistrict'] = df.Month.astype('str') + df.PdDistrict
    return df

In [4]:
def get_data(cols_x, cols_y):
    df = pd.DataFrame.from_csv('train.csv.gz')
    # df = df.sample(df.shape[0])
    df.reset_index(inplace=True)
    df = trata_vars(df)
    features_x = []
    X = np.array(pd.get_dummies(df[cols_x]))

    features_y = []
    Y = np.array(pd.get_dummies(df[cols_y]))
    
    return X, Y, df

In [5]:
cols_x = ['DayOfWeek', 'PdDistrict',  'esquina', 'Year', 'Month', 'Day','YearMonth','YearDistrict','MonthDistrict']
cols_y = ['Category']

In [6]:
X, Y, df = get_data(cols_x,cols_y)

In [7]:
# df_cat = pd.DataFrame(df.Category.unique(), columns=['Category'])
# df_cat['IndCat'] = list(df_cat.index)
# df = df.merge(df_cat, how='left', on='Category')
# Y = np.array(df['IndCat'])

In [8]:
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor, ExtraTreesRegressor, ExtraTreesClassifier
from sklearn.cross_validation import KFold, train_test_split
from sklearn.metrics import log_loss

In [9]:
# qtde = X_train.shape[0]
# # qtde = 200000

# if qtde != X_train.shape[0]:
#     X = np.array(X_train.iloc[:qtde])
#     Y = np.array(Y_train.iloc[:qtde])
# else:
#     X = np.asfortranarray(X_train)
#     Y = np.asfortranarray(Y_train)

In [11]:
def train_forest(X, Y):
    ini = datetime.now()
    train_x, test_x, train_y, test_y = train_test_split(X, Y, test_size=0.01)
    
    forest = ExtraTreesClassifier(n_estimators=80,
                            random_state=500,
                            n_jobs=4,
                           max_leaf_nodes=80,
                            warm_start=True,
                            min_samples_leaf=200,
                            max_features=100#X.shape[1]
                           )

    kf = KFold(train_x.shape[0], n_folds=5, shuffle=True, random_state=500)
    print('Training forest...')
    for k, (train, test) in enumerate(kf, start=1):
        forest.fit(train_x[train], train_y[train])
        pred = forest.predict(train_x[test])
        print('fold:', k, 'log loss:', log_loss(train_y[test], pred), 'accuracy:', forest.score(train_x[test],train_y[test]))
    print('Testing...')
    pred = forest.predict(test_x)
    print('fold:', k, 'log loss:', log_loss(test_y, pred), 'accuracy:', forest.score(test_x,test_y))
    fim = datetime.now()
    print(fim - ini)
    return forest

In [ ]:
forest = train_forest(X, Y)

In [ ]:
# import pickle
# with open('temp', 'wb') as f:
#     pickle.dump(forest, f)

In [ ]:
# import pickle
# with open('temp', 'rb') as f:
#     forest = pickle.load(f)

In [ ]:
def get_test():
    df_test = pd.read_csv('test.csv.gz', index_col='Id')
    df_test.reset_index(inplace=True)
    df_test.Dates = pd.to_datetime(df_test.Dates)
    df_test = trata_vars(df_test)

    X_test = pd.get_dummies(df_test[cols_x])
    return X_test

In [ ]:
X_test = get_test()

In [ ]:
# a = X_train.columns - X_test.columns
# for i in a:
#     X_test[i] = 0

In [ ]:
# res = forest.predict(np.array(X_test))


# X_test.drop(X_test.columns - X_train.columns, axis=1, inplace=True)
import gc
l = []
for i in range(0, X_test.shape[0],100000):
    print(i)
    res = forest.predict(np.array(X_test.iloc[i:i+100000]))
    l.append(res)
    del res
    gc.collect()

In [ ]:
newl = []
for i in l:
    for j in i:
        newl.append(j)
res = np.array(newl)

In [ ]:
cols = df['Category'].unique()
cols.sort()

res = pd.DataFrame(res)
res.columns = cols
res['Id'] = res.index
res.set_index('Id', inplace=True)

In [ ]:
today = datetime.today()
t = today.strftime('%Y%m%d%H%M')
res.to_csv('result_%s.csv.gz'%t, compression='gzip')